# Implementation of the Q-Learning Algorithm for Atari Games

In [1]:
import os
import gym
import math
import random
import numpy as np
from itertools import count
from collections import namedtuple
import torch
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Started on device:", device)
print(torch.__version__)

Started on device: cpu
0.4.0


In [2]:
# this method calculates in which episodes a video should be recorded
def video_callable(episode_id):
    return episode_id % 20 == 0

In [3]:
# the image processor to crop and resize an image and convert it to grayscale
class ImageProcessor:

    def __init__(self, size, device):
        self.transformations = T.Compose([T.ToPILImage(),
                                          T.Grayscale(),
                                          T.Resize(size),
                                          T.ToTensor()])
        self.device = device

    def process(self, screen):
        # crop the image to a square image
        screen = screen[34:-16, :, :]
        return self.transformations(screen).squeeze(0).to(self.device)

In [4]:
# a single transition from one state into the next state with a given action
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

In [5]:
# the experience replay memory to save transitions and sample a random batch from them
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [6]:
# the DQN architecture with the forward function
class DQN(nn.Module):

    def __init__(self, in_channels, num_actions):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc4 = nn.Linear(3136, 512)
        self.fc5 = nn.Linear(512, num_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc4(x.view(x.size(0), -1)))
        return self.fc5(x)

In [7]:
# the dueling DQN architecture with the forward function
# https://arxiv.org/abs/1511.06581
class DuelingDQN(nn.Module):

    def __init__(self, in_channels, num_actions):
        super(DuelingDQN, self).__init__()
        self.num_actions = num_actions

        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        self.fc1_adv = nn.Linear(3136, 512)
        self.fc1_val = nn.Linear(3136, 512)

        self.fc2_adv = nn.Linear(512, num_actions)
        self.fc2_val = nn.Linear(512, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = x.view(x.size(0), -1)

        adv = self.relu(self.fc1_adv(x))
        val = self.relu(self.fc1_val(x))

        adv = self.fc2_adv(adv)
        val = self.fc2_val(val).expand(x.size(0), self.num_actions)

        x = val + adv - adv.mean(1).unsqueeze(1).expand(x.size(0), self.num_actions)
        return x

### Create the Gym Environment:

https://gym.openai.com/docs/

In [8]:
# Atari Actions: 0 (noop), 1 (fire), 2 (left) and 3 (right) are valid actions for Breakout
env_name = "MsPacman-v0"
env = gym.make(env_name)

#start video recording
video_path = os.getcwd() + "/recording-" + env_name
env = gym.wrappers.Monitor(env, video_path, force=True)

env.reset()

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[228, 111, 111],
        [228, 111, 111],
        [228, 111, 111],
        ..., 
        [228, 111, 111],
        [228, 111, 111],
        [228, 111, 111]],

       [[228, 111, 111],
        [228, 111, 111],
        [228, 111, 111],
        ..., 
        [228, 111, 111],
        [228, 111, 111],
        [228, 111, 111]],

       ..., 
       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ..., 
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,

### Set up the hyperparameters for the training:

In [9]:
BATCH_SIZE = 32

# the discount factor of the MDP
GAMMA = 0.99

# parameters for the exploration strategy
EPS_START = 1.0
EPS_END = 0.02
EPS_DECAY = 100000

# update the DQN network every 1000 steps
TARGET_UPDATE = 1000

# the input channels of the DQN
IN_CHANNELS = 4

# sample at the beginning 10000 Transitions with random actions for the replay memory
LEARNING_STARTS = 10000

# number of training episodes (= full games)
num_episodes = 10000

# just for observation
steps_done = 0
steps_trained = 0

### Create the DQN, image processor, optimizer and replay memory:

In [10]:
# 84 is the used size in DQN paper
image_processor = ImageProcessor(84, device)

# create the two networks for the iterative update
policy_net = DuelingDQN(IN_CHANNELS, env.action_space.n).float().to(device)
target_net = DuelingDQN(IN_CHANNELS, env.action_space.n).float().to(device)

# load the state of the polciy net into the target net, the target net gets updated each  TARGET_UPDATE steps
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=0.0001)
memory = ReplayMemory(100000)

### Select an action:

This method selects with an probality a random action or the DQN will choose the next action. `eps_threshold` should get smaller over time and at the end only the DQN should choose the next actions. 

In [11]:
def select_action(current_state):
    global steps_done
    global steps_trained
    sample = random.random()
    
    # calculate the epsilon for the exploration
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_trained / EPS_DECAY)
    if LEARNING_STARTS < steps_done:
        steps_trained += 1

    steps_done += 1
    if LEARNING_STARTS < steps_done and sample > eps_threshold:
        with torch.no_grad():
            return policy_net(current_state).max(1)[1].view(-1, 1)
    else:
        return torch.LongTensor([[random.randrange(env.action_space.n)]]).to(device)

### Optimize the model:

This method will optimize the model. It samples from the memory the batch size which will be used for training. The zip function is used to transpose the batch in the correct format (https://www.programiz.com/python-programming/methods/built-in/zip). Double DQN is used as a standard because the training is faster. 

In [12]:
def optimize_model(double_dqn=True):
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    # Transpose the batch
    batch = Transition(*zip(*transitions))
    state_batch = Variable(torch.from_numpy(np.asarray(batch.state)).to(device))
    action_batch = Variable(torch.from_numpy(np.asarray(batch.action)).long().to(device))
    reward_batch = Variable(torch.from_numpy(np.asarray(batch.reward)).to(device))
    done_batch = Variable(torch.from_numpy(np.asarray(batch.done)).float().to(device))
    next_state_batch = Variable(torch.from_numpy(np.asarray(batch.next_state)).to(device))

    # get the right q values for the selected actions
    q_s_a = policy_net(state_batch).gather(1, action_batch.unsqueeze(1))
    q_s_a = q_s_a.squeeze()

    if double_dqn:
        # get the Q values for best actions in next_state_batch
        # based off the current Q network
        # max(Q(s', a', theta_i)) wrt a'
        q_next_state_values = policy_net(next_state_batch).detach()
        _, a_prime = q_next_state_values.max(1)
        # get Q values from frozen network for next state and chosen action
        # Q(s',argmax(Q(s',a', theta_i), theta_i_frozen)) (argmax wrt a')
        q_target_next_state_values = target_net(next_state_batch).detach()
        q_target_s_a_prime = q_target_next_state_values.gather(1, a_prime.unsqueeze(1))
        q_target_s_a_prime = q_target_s_a_prime.squeeze()

        # if current state is end of episode, then there is no next Q value
        q_target_s_a_prime = (1 - done_batch) * q_target_s_a_prime

        error = (reward_batch + (GAMMA * q_target_s_a_prime)) - q_s_a
    else:
        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the columns of actions taken
        next_state_values = target_net(next_state_batch).detach()
        q_s_a_prime, a_prime = next_state_values.max(1)
        q_s_a_prime = (1 - done_batch) * q_s_a_prime

        # Compute Bellman error
        # r + gamma * Q(s',a', theta_i_frozen) - Q(s, a, theta_i)
        error = (reward_batch + (GAMMA * q_s_a_prime)) - q_s_a

    # clip the error and flip
    clipped_error = -1.0 * error.clamp(-1, 1)
    # Optimize the model
    optimizer.zero_grad()

    q_s_a.backward(clipped_error.data)

    optimizer.step()

### Start the training of the agent:

In [13]:
for i_episode in range(num_episodes):
    print("Episode: ", i_episode)
    # Initialize the environment and state
    state = env.reset()
    state = image_processor.process(state)
    
    # stack the first image 4 times as it is the only one
    state = np.stack([state]*4, axis=0)
    for t in count():
        # Select and perform an action
        action = select_action(torch.FloatTensor(state).unsqueeze(0).to(device))
        next_state, reward, done, _ = env.step(action.item())
        reward = torch.Tensor([reward], device=device)
        print("Reward:", reward.item(), "Episode:", i_episode, "Steps done:", steps_done)
        # Observe new state
        next_state = image_processor.process(next_state)
        
        # append the new pixels from the environment
        next_state = np.append(state[1:, :, :], np.expand_dims(next_state, 0), axis=0)

        if done is False:
            done = 0
        else:
            done = 1

        # Store the transition in memory
        memory.push(state, action, next_state, reward, done)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()

        # Update the target network
        if steps_done % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if done:
            print("Done")
            break

    torch.save(target_net.state_dict(), os.getcwd() + '/' + env_name + '_dueling_ddqn')

print('Complete')
env.close()

Episode:  0
Reward: 0.0 Episode: 0 Steps done: 1
Reward: 0.0 Episode: 0 Steps done: 2
Reward: 0.0 Episode: 0 Steps done: 3
Reward: 0.0 Episode: 0 Steps done: 4
Reward: 0.0 Episode: 0 Steps done: 5
Reward: 0.0 Episode: 0 Steps done: 6
Reward: 0.0 Episode: 0 Steps done: 7
Reward: 0.0 Episode: 0 Steps done: 8
Reward: 0.0 Episode: 0 Steps done: 9
Reward: 0.0 Episode: 0 Steps done: 10
Reward: 0.0 Episode: 0 Steps done: 11
Reward: 0.0 Episode: 0 Steps done: 12
Reward: 0.0 Episode: 0 Steps done: 13
Reward: 0.0 Episode: 0 Steps done: 14
Reward: 0.0 Episode: 0 Steps done: 15
Reward: 0.0 Episode: 0 Steps done: 16
Reward: 0.0 Episode: 0 Steps done: 17
Reward: 0.0 Episode: 0 Steps done: 18
Reward: 0.0 Episode: 0 Steps done: 19
Reward: 0.0 Episode: 0 Steps done: 20
Reward: 0.0 Episode: 0 Steps done: 21
Reward: 0.0 Episode: 0 Steps done: 22
Reward: 0.0 Episode: 0 Steps done: 23
Reward: 0.0 Episode: 0 Steps done: 24
Reward: 0.0 Episode: 0 Steps done: 25
Reward: 0.0 Episode: 0 Steps done: 26
Reward: 0

KeyboardInterrupt: 